In [20]:
#전체 기사 내용에서 쓰이는 단어들의 tf-idf 값을 구해서 자주 쓰이는 쓸모없는 단어들을 수치로 표현해준다.
import sys
from konlpy.tag import Twitter
from konlpy.tag import Mecab
from collections import Counter
import os
import math
import re

In [21]:
#tfidf를 구하기 위해서 tf와 idf를 구하기 위한 시작 1글자 이상의 단어만
# 생각해볼점 한글자인 것들을 제거 한후 할 것인지 아니면 1글자들도 tf-idf 를 구해서 제거해서 할 것인지
# 제거 한 후 하는 것은 한 글자들은 대체로 의미가 없는 단어들이지만 필요한 단어가 있을 수도 있으며 좋은 점은 tf-idf 후 글자 clean의 시간을 단축시킴
# 제거 하지 않으면 실제로 필요한 단어들도 존재 할 수 있으며 자주쓰이는 단어는 당연히 골라 낼 수 있을 것이다. 하지만 clean 시간이 길어 진다.
def get_tags(text):
    #spliter = Twitter()
    spliter = Mecab()
    nouns = spliter.nouns(text)
    count = Counter(nouns)
    return_list = []
    for n, c in count.most_common():
        if len(n) > 1:
            temp = {'tag': n, 'count': c, 'dict_count' : 1}
            return_list.append(temp)
        #temp = {'tag': n, 'count': c, 'dict_count' : 1}
        #return_list.append(temp)
    return return_list

In [31]:
#tfidf가 0에 가까울 수록 좋지 제거 대상
#파라미터로는 각각의 단어에 대한 단어의 총 count와 어떤 문서에 쓰여져있는지의 count를 저장한 dict를 가져오고 총 문서의 수인 dict_num을 가져온다.
#많은 문서에서 쓰이는 단어 일수록 idf 값이 0에 가까워 지며 그로 인해 tf-idf 값이 0에 가까워 지면 제거 대상이다.
#로그빈도를 이용해서 tf값을 구했다. 값이 커지는 것을 막을 수 있다.
def tfidf_func(result_dict, dict_num):
    result = []
    for tag in result_dict:
        tf = math.log10(result_dict[tag][0] + 1)
        idf = math.log10(dict_num / result_dict[tag][1])
        tfidf = tf * idf
        #print(tag,result_dict[tag][0],result_dict[tag][1],tfidf)
        #tfidf 값을 조절해서 자주쓰이는 단어에 대해 알아낼수 있다.
        if tfidf < 0.9:
            result.append(tag)
    #print(dict_num)
    #print(result)
    return result

In [32]:
#tfidf 하기
#처음에는 각각의 문서에 쓰여져 있는 단어의 count와 문서 count값을 가져와서 diction에 저장하고 계속해서 
def result_tfidf():
    num = 0
    #전체 문서에 쓰인 명사와 그 명사의 count, 명사의 문서에 쓰인 count 수를 저장하기 위한 변수
    result_dict = {}
    
    for file in os.listdir("cleand_article_dict"):
        #존재 여부 확인
        if file.endswith(".txt"):
            open_text_file = open('cleand_article_dict/'+file, 'r')
            text = open_text_file.read()
            
            #get_tags(text)를 통해 하나의 문서에 쓰인 명사와 그 명사의 count수 그리고 문서 count수인 1을 return 받는다.
            tags = get_tags(text)
            open_text_file.close()
            
            #전체적인 명사와 count, dict_count를 저장해 result_dict에 그 값들을 저장한다. 
            for tag in tags:
                noun = tag['tag']
                count = tag['count']
                dict_count = tag['dict_count']
                if noun in result_dict:
                    result_dict[noun][0] += count
                    result_dict[noun][1] += dict_count
                else:
                    result_dict[noun] = [count, dict_count]
                    
            #총 문서의 수를 저장하기 위해 사용하는 변수 하나의 문서를 처리할떄 마다 값을 1증가 시킨다.        
            num = num + 1
            
    #위에서 계산한 result_dict와 전체 문서의 수인 num값을 파라미터로 넘겨준다.        
    return tfidf_func(result_dict, num)  

In [29]:
result_tfidf()

전체 391 271 3.0705312255238297
회의 1250 672 2.4456905975317667
북한 5897 883 2.530292360946424
미사일 3660 1065 2.1012791934871093
발사 2411 523 3.0390614485793495
국회 3026 1428 1.6091750392047648
본문 8240 4139 0.00041084393676191623
내용 9524 4139 0.00041744150630364956
플레이어 8278 4139 0.0004110535530629761
정보 1493 1129 1.7913063554490412
국정원 16 5 3.5904872804460375
업무 528 497 2.5073329940548104
보고 98 53 3.7771875690415793
예정 939 681 2.3304958688927124
국방부 184 132 3.3926556501950187
동향 20 16 3.190356965846884
태세 114 78 3.5545095299971154
뉴스 3384 2194 0.9733242221557035
경제 1299 771 2.2730103792448575
오류 4139 4139 0.00037947657853131987
우회 4146 4139 0.00037955355385750427
함수 4139 4139 0.00037947657853131987
추가 4355 4139 0.0003817938855559326
국방위원회 50 43 3.3870198910896505
오전 1735 1448 1.4779855331395428
소집 162 103 3.548709761576893
대륙 671 319 3.1474547816761116
간탄 497 277 3.167951350625587
성공 924 588 2.514169998554349
발표 3486 1422 1.6440545869373386
관련 3894 1780 1.3162094419301384
긴급 61 52 3.40733122

축사 12 11 2.8690810265102145
조건 324 309 2.8309976995209976
이슈 689 416 2.8329074093816677
표현 99 68 3.5689828568293254
재제 1 1 1.0888255970042429
명기 4 4 2.107352869367607
조성 164 103 3.557205954552017
주말 7 7 2.5032772130493357
인순 1 1 1.0888255970042429
저작 342 342 2.745658210317513
복제 73 73 3.2780295736639444
박영태 87 84 3.291466760346011
추미 38 36 3.2787033995625747
영등포구 106 103 3.255462605215026
여의도 542 511 2.484782900376952
오바마 82 23 4.328044056881862
머리 46 40 3.3691774427570147
이명박 41 26 3.5744387540678915
재임 9 8 2.7139103541289553
일곱 1 1 1.0888255970042429
친밀 1 1 1.0888255970042429
나이 18 15 3.1213212782893613
친구 199 196 3.048276301311914
모양 8 8 2.589728626717381
시선 14 13 2.943823445462453
염색 4 3 2.1946812986364472
후문 2 2 1.5821199318375971
청년 217 81 3.9952887497203515
고용 707 365 3.005951385627665
국정 297 109 3.9082072931060736
기획 255 150 3.470054837810269
기업 1143 1030 1.8477881982629059
대변인 273 144 3.5557952374133968
비율 49 30 3.6355903807016645
민간 86 47 3.7721758106034184
취업 425 355 2.80498

한계 18 17 3.051811221854968
코앞 2 2 1.5821199318375971
피로감 2 2 1.5821199318375971
기조 73 51 3.5691674310720742
결단 54 45 3.4177030647951687
위기관리 28 27 3.1962705542838274
상황실 33 24 3.425595347615265
전환 342 307 2.864532397956471
지하 9 8 2.7139103541289553
벙커 6 6 2.3991058024783096
평북 20 17 3.155544335058327
재원 14 14 2.9059713206738733
제제 3 3 1.890395575529579
당사자 274 273 2.8804559932768306
태도 63 58 3.347885254489492
지난주 43 41 3.2938349455468297
자제 299 298 2.8308147268024797
며칠 311 305 2.8251301601469
실망 55 44 3.450132398040451
유감 76 57 3.5109959666805537
단합 34 28 3.350384171808451
망상 43 39 3.3295295045864504
달성 46 45 3.2836454195447398
능동 18 18 3.020067974778029
상임 64 58 3.360366066391682
정오 25 19 3.308556818317465
기점 2 2 1.5821199318375971
구상 101 59 3.7081738516890037
이야기 163 98 3.6008361300983664
파스 21 21 3.0805661241907965
연예뉴스 21 21 3.0805661241907965
파이낸셜 21 21 3.0805661241907965
팔장 9 7 2.771902301106642
기념 42 42 3.256729449634674
촬영 25 24 3.1649969606957975
곽상 13 10 2.999417460338175
처분

위업 2 2 1.5821199318375971
장애 16 15 3.0034139472095798
안병준 2 2 1.5821199318375971
박태인 2 2 1.5821199318375971
개정안 89 47 3.8008111093545836
착수 41 34 3.3853211717452774
공전 3 3 1.890395575529579
정국 41 29 3.497456724724673
공청회 12 10 2.9151901329763494
소위 44 34 3.447810116033389
주무 7 7 2.5032772130493357
위인 2 2 1.5821199318375971
국토 300 288 2.8692080403785414
산자위 3 3 1.890395575529579
환노위 22 15 3.3238538422363577
최건 2 2 1.5821199318375971
임성호 12 6 3.1623170728290733
민구 2 2 1.5821199318375971
강경 289 287 2.8542109367860715
건데요 6 6 2.3991058024783096
동결 294 283 2.877870957366405
재개 57 37 3.612908834485887
이영훈 3 2 1.9964130774295725
단칼 6 6 2.3991058024783096
환부 6 6 2.3991058024783096
희생 10 8 2.8262463909650624
골단 10 10 2.7253250122998844
각오 43 34 3.4274557662713323
스스로 17 17 2.9957723792181765
연예 27 27 3.162961525945747
조이 20 20 3.0622206767947175
라이브 28 26 3.2202398661021365
캐스트 20 20 3.0622206767947175
박지환 5 5 2.2706689552265593
전격 16 13 3.079883772034613
당초 285 282 2.8659680973651342
성능 17 14 

송구 1 1 1.0888255970042429
대대 13 12 2.9086656143430547
장영근 4 4 2.107352869367607
항공대 4 4 2.107352869367607
안정식 2 2 1.5821199318375971
동영상 8 7 2.6450670077286413
퇴장 11 7 2.9913849792305447
재작년 3 3 1.890395575529579
상사 13 11 2.95197614340633
연세대 40 25 3.5788646812606526
합격 27 21 3.320910821450819
연구원 78 44 3.7450575143638107
브루스 6 3 2.65350566179944
클링 7 3 2.835593363294368
헤리티지 6 3 2.65350566179944
스웨덴 7 3 2.835593363294368
東京 2 2 1.5821199318375971
新聞 1 1 1.0888255970042429
스톡홀름 3 3 1.890395575529579
열리 2 2 1.5821199318375971
제의 13 13 2.868823789780146
적대 9 9 2.662757831681574
불만 17 14 3.10161806864339
자신감 8 8 2.589728626717381
박광수 4 4 2.107352869367607
문요한 3 2 1.9964130774295725
도심 262 261 2.904817680018457
심장 5 4 2.3460796030234734
프로젝트 6 4 2.5479201803699087
발의 10 6 2.956356677361856
국회의원 41 27 3.5478330226211194
부영 1 1 1.0888255970042429
올인 2 2 1.5821199318375971
김춘상 2 2 1.5821199318375971
선도 6 6 2.3991058024783096
별도 29 25 3.277821182565493
구심점 2 2 1.5821199318375971
천우신조 1 1 1.088

...

면접시험 2 2 1.5821199318375971
高角 1 1 1.0888255970042429
통용 2 2 1.5821199318375971
양욱 1 1 1.0888255970042429
스팅 1 1 1.0888255970042429
운반체 1 1 1.0888255970042429
외양 1 1 1.0888255970042429
이일우 1 1 1.0888255970042429
자주국방 1 1 1.0888255970042429
륜형 1 1 1.0888255970042429
크기 7 7 2.5032772130493357
길이 5 4 2.3460796030234734
동체 2 2 1.5821199318375971
미지 1 1 1.0888255970042429
난도 1 1 1.0888255970042429
전자전 1 1 1.0888255970042429
성주 13 7 3.1769549394593435
김준범 6 6 2.3991058024783096
골프장 8 7 2.6450670077286413
영실 1 1 1.0888255970042429
대구경북 3 2 1.9964130774295725
적발 1 1 1.0888255970042429
지향 263 262 2.9027743261336085
청렴도 1 1 1.0888255970042429
자구책 1 1 1.0888255970042429
외계 1 1 1.0888255970042429
이준기 4 2 2.317763806742132
관람 14 1 4.253922485193707
경복궁 10 1 3.7667176974581094
창경궁 9 1 3.617000341120899
예매 8 1 3.4514954821541006
고궁 6 1 3.0567198990121702
휴무 5 1 2.814573338081293
관람권 4 1 2.5281747441166567
한복 4 1 2.5281747441166567
티켓 4 1 2.5281747441166567
화요일 5 3 2.443300837136575
인터 5 2 2.580326470

부적합 2 2 1.5821199318375971
여타 1 1 1.0888255970042429
향배 2 2 1.5821199318375971
스카이 2 1 1.7257477410770503
초유 1 1 1.0888255970042429
홀대 2 2 1.5821199318375971
괄목 1 1 1.0888255970042429
득세 1 1 1.0888255970042429
장관순 1 1 1.0888255970042429
명마 1 1 1.0888255970042429
마리 3 2 1.9964130774295725
송환 1 1 1.0888255970042429
모친 1 1 1.0888255970042429
월시 1 1 1.0888255970042429
트로이카 1 1 1.0888255970042429
삼두마차 1 1 1.0888255970042429
교감 2 2 1.5821199318375971
가교 2 2 1.5821199318375971
피비 1 1 1.0888255970042429
모피아 2 2 1.5821199318375971
집필 1 1 1.0888255970042429
공정위 1 1 1.0888255970042429
막중 1 1 1.0888255970042429
이용섭 1 1 1.0888255970042429
최동현 1 1 1.0888255970042429
암묵 1 1 1.0888255970042429
가이드라인 258 258 2.9089448024541116
배후 1 1 1.0888255970042429
고원 1 1 1.0888255970042429
순수 3 3 1.890395575529579
압수 1 1 1.0888255970042429
노트 1 1 1.0888255970042429
귀엣말 1 1 1.0888255970042429
스모킹 1 1 1.0888255970042429
쌍방 1 1 1.0888255970042429
기법 1 1 1.0888255970042429
파괴력 1 1 1.0888255970042429
칼날 1 1 1.088825597

체설 1 1 1.0888255970042429
도미노 2 2 1.5821199318375971
이직 1 1 1.0888255970042429
승산 1 1 1.0888255970042429
일로 1 1 1.0888255970042429
폭풍우 1 1 1.0888255970042429
할부 4 1 2.5281747441166567
면제 2 1 1.7257477410770503
제휴 2 1 1.7257477410770503
대리점 1 1 1.0888255970042429
판매점 1 1 1.0888255970042429
한시 2 2 1.5821199318375971
장도 3 3 1.890395575529579
고지 2 2 1.5821199318375971
무이자 1 1 1.0888255970042429
동통 1 1 1.0888255970042429
신단 1 1 1.0888255970042429
데당 1 1 1.0888255970042429
서독 1 1 1.0888255970042429
동질 1 1 1.0888255970042429
드레스 1 1 1.0888255970042429
사견 1 1 1.0888255970042429
이정하 1 1 1.0888255970042429
인시 1 1 1.0888255970042429
김세진 3 1 2.1776511940084857
대보 1 1 1.0888255970042429
판명 10 10 2.7253250122998844
이사국 13 10 2.999417460338175
세계만방 11 11 2.7795474794916006
크워스 1 1 1.0888255970042429
복선 1 1 1.0888255970042429
디스커버리 1 1 1.0888255970042429
함북 1 1 1.0888255970042429
화대군 1 1 1.0888255970042429
안변군 1 1 1.0888255970042429
대령 1 1 1.0888255970042429
서방 1 1 1.0888255970042429
몸값 2 2 1.58211993

최수용 3 1 2.1776511940084857
허풍 1 1 1.0888255970042429
홍대표 4 1 2.5281747441166567
분전 1 1 1.0888255970042429
착석 1 1 1.0888255970042429
보습 1 1 1.0888255970042429
추대 1 1 1.0888255970042429
나자 2 2 1.5821199318375971
이상규 2 2 1.5821199318375971
티브로드 1 1 1.0888255970042429
강신웅 1 1 1.0888255970042429
하도급 1 1 1.0888255970042429
권순종 1 1 1.0888255970042429
김봉 1 1 1.0888255970042429
야만 2 1 1.7257477410770503
이승환 1 1 1.0888255970042429
위령 1 1 1.0888255970042429
안병욱 1 1 1.0888255970042429
학살 1 1 1.0888255970042429
분단 1 1 1.0888255970042429
강군 2 2 1.5821199318375971
정예 1 1 1.0888255970042429
텔레비 1 1 1.0888255970042429
김형구 2 1 1.7257477410770503
윤유 1 1 1.0888255970042429
신상철 1 1 1.0888255970042429
서프라이즈 1 1 1.0888255970042429
집행유예 1 1 1.0888255970042429
향응 11 3 3.388498624901364
적선 2 2 1.5821199318375971
주광덕 3 2 1.9964130774295725
접대 6 3 2.65350566179944
좌담회 4 1 2.5281747441166567
정현수 2 1 1.7257477410770503
무자 2 1 1.7257477410770503
일맥상통 1 1 1.0888255970042429
삼각 1 1 1.0888255970042429
편대 1 1 1.08882559

['본문',
 '내용',
 '플레이어',
 '오류',
 '우회',
 '함수',
 '추가',
 '기자',
 '무단',
 '전재',
 '배포',
 '금지',
 '서울']

In [33]:
#tfidf를 통해 얻어온 제거 대상을 clean해주는 함수.
remove_list = '|'.join(result_tfidf())

def clean_text(text):
    cleaned_text = re.sub(remove_list, '', text)
    return cleaned_text

In [34]:
#main function 각 문서들의 최종 clean된 명사들을 word2vec하기위해 하나의 txt파일에 저장한다.
def main():
    spliter = Twitter()
    #입력 파일명 = file
    for file in os.listdir("cleand_article_dict"):
        #존재 여부 확인
        if file.endswith(".txt"):
            #출력 파일명
            output_file_name = 'nouns.txt'
            #output_file_name = 'konlpy/konlpy'+str(num)+'.txt'
            
            open_text_file = open('cleand_article_dict/'+file, 'r')
            text = open_text_file.read()
            open_text_file.close()
            
            #open_output_file = open(output_file_name, 'w')
            open_output_file = open(output_file_name, 'a+')
            
            result_text = clean_text(text)
            nouns = spliter.nouns(result_text)
            
            for n in nouns:
                if len(n) > 1:
                    open_output_file.write(' ' + n)
            open_output_file.close()

In [35]:
if __name__ == '__main__':
    main()